In [1]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

my_model = [0,0,0]

data_path = 'f_images/front/'

onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path,f))]

Training_Data, Labels = [], []

for i, files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]

    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    if images is None:
        continue

    Training_Data.append(np.asarray(images, dtype=np.uint8))

    Labels.append(i)


if len(Labels) == 0:
    print("There is no data to train.")
    exit()

Labels = np.asarray(Labels, dtype=np.int32)

my_model[0] = cv2.face.LBPHFaceRecognizer_create()

my_model[0].train(np.asarray(Training_Data), np.asarray(Labels))


#print(my_model.train(np.asarray(Training_Data), np.asarray(Labels)))
print("Front face Model Training Completed!!!!!")

Front face Model Training Completed!!!!!


In [2]:
data_path = 'f_images/left/'

onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path,f))]

Training_Data, Labels = [], []

for i, files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]

    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    if images is None:
        continue

    Training_Data.append(np.asarray(images, dtype=np.uint8))

    Labels.append(i)


if len(Labels) == 0:
    print("There is no data to train.")
    exit()

Labels = np.asarray(Labels, dtype=np.int32)

my_model[1] = cv2.face.LBPHFaceRecognizer_create()

my_model[1].train(np.asarray(Training_Data), np.asarray(Labels))


#print(my_model.train(np.asarray(Training_Data), np.asarray(Labels)))
print("Left face Model Training Completed!!!!!")

Left face Model Training Completed!!!!!


In [3]:
data_path = 'f_images/right/'

onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path,f))]

Training_Data, Labels = [], []

for i, files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]

    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    if images is None:
        continue

    Training_Data.append(np.asarray(images, dtype=np.uint8))

    Labels.append(i)


if len(Labels) == 0:
    print("There is no data to train.")
    exit()

Labels = np.asarray(Labels, dtype=np.int32)

my_model[2] = cv2.face.LBPHFaceRecognizer_create()

my_model[2].train(np.asarray(Training_Data), np.asarray(Labels))


#print(my_model.train(np.asarray(Training_Data), np.asarray(Labels)))
print("Right face Model Training Completed!!!!!")

Right face Model Training Completed!!!!!


In [4]:
#python run_video_face_detect_copy_test.py --net_type slim --test_device cpu --input_size 320

import argparse
import sys
import cv2

import time

import copy

from vision.ssd.config.fd_config import define_img_size


###성능 테스트###
nowtime = time.time()
face_count = 0
total_frame = 0


###############################
#input_img_size = args.input_size
input_img_size = 320
threshold = 0.6
candidate_size = 1000

define_img_size(input_img_size)  # must put define_img_size() before 'import create_mb_tiny_fd, create_mb_tiny_fd_predictor'

from vision.ssd.mb_tiny_fd import create_mb_tiny_fd, create_mb_tiny_fd_predictor
from vision.ssd.mb_tiny_RFB_fd import create_Mb_Tiny_RFB_fd, create_Mb_Tiny_RFB_fd_predictor
from vision.utils.misc import Timer

label_path = "./models/voc-model-labels.txt"

net_type = 'slim'




videoFile = 0
cap = cv2.VideoCapture(0) #영상 장치 인덱스에 따라 작동

width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

fps = cap.get(cv2.CAP_PROP_FPS)



fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('self.mp4', fourcc, 7, (int(width), int(height))) #결과 영상을 self.mp4로 저장
#내꺼 7

################################
class_names = [name.strip() for name in open(label_path).readlines()]
num_classes = len(class_names)
#test_device = args.test_device
test_device = 'cpu'

#candidate_size = args.candidate_size
#threshold = args.threshold







if net_type == 'slim':
    model_path = "models/pretrained/version-slim-320.pth"
    # model_path = "models/pretrained/version-slim-640.pth"
    net = create_mb_tiny_fd(len(class_names), is_test=True, device=test_device)
    predictor = create_mb_tiny_fd_predictor(net, candidate_size=candidate_size, device=test_device)
elif net_type == 'RFB':
    model_path = "models/pretrained/version-RFB-320.pth"
    # model_path = "models/pretrained/version-RFB-640.pth"
    net = create_Mb_Tiny_RFB_fd(len(class_names), is_test=True, device=test_device)
    predictor = create_Mb_Tiny_RFB_fd_predictor(net, candidate_size=candidate_size, device=test_device)
else:
    print("The net type is wrong!")
    sys.exit(1)
net.load(model_path)

timer = Timer()


sum = 0




###############
##여기 수정함##
###############



def get_circle_x(x1, y1, x2, y2, y):
    a = x2 - x1
    b = y2 - y1
    x = ((x2+x1)/2 + ((x2-x1)/2)*(1-((y-(y2+y1)/2)**2)/((y2-y1)/2)**2)**0.5,(x2+x1)/2 - ((x2-x1)/2)*(1-((y-(y2+y1)/2)**2)/((y2-y1)/2)**2)**0.5 )
#    print(x)
    return x

def blur_circle(img, x1, y1, x2, y2):

    img_blur = copy.deepcopy(img)

    for i in range(int(img_blur.shape[0]/10)):
        for j in range(int(img_blur.shape[1]/10)):
            img_blur[i*10:(i+1)*10,j*10:(j+1)*10] = cv2.blur(img_blur[i*10:(i+1)*10,j*10:(j+1)*10],(50,50))

    yi = y1
    while True:
        if yi == y2:
            break
        img[yi, round(get_circle_x(x1, y1, x2, y2, yi)[1]):round(get_circle_x(x1, y1, x2, y2, yi)[0])] = img_blur[yi, round(get_circle_x(x1, y1, x2, y2, yi)[1]):round(get_circle_x(x1, y1, x2, y2, yi)[0])]
        yi += 1

    return img



mouse_event_types = { 0:"EVENT_MOUSEMOVE", 1:"EVENT_LBUTTONDOWN", 2:"EVENT_RBUTTONDOWN", 3:"EVENT_MBUTTONDOWN",
                 4:"EVENT_LBUTTONUP", 5:"EVENT_RBUTTONUP", 6:"EVENT_MBUTTONUP",
                 7:"EVENT_LBUTTONDBLCLK", 8:"EVENT_RBUTTONDBLCLK", 9:"EVENT_MBUTTONDBLCLK",
                 10:"EVENT_MOUSEWHEEL", 11:"EVENT_MOUSEHWHEEL"}

mouse_event_flags = { 0:"None", 1:"EVENT_FLAG_LBUTTON", 2:"EVENT_FLAG_RBUTTON", 4:"EVENT_FLAG_MBUTTON",
                8:"EVENT_FLAG_CTRLKEY", 9:"EVENT_FLAG_CTRLKEY + EVENT_FLAG_LBUTTON",
                10:"EVENT_FLAG_CTRLKEY + EVENT_FLAG_RBUTTON", 11:"EVENT_FLAG_CTRLKEY + EVENT_FLAG_MBUTTON",

                16:"EVENT_FLAG_SHIFTKEY", 17:"EVENT_FLAG_SHIFTKEY + EVENT_FLAG_LBUTTON",
                18:"EVENT_FLAG_SHIFTLKEY + EVENT_FLAG_RBUTTON", 19:"EVENT_FLAG_SHIFTKEY + EVENT_FLAG_MBUTTON",

                32:"EVENT_FLAG_ALTKEY", 33:"EVENT_FLAG_ALTKEY + EVENT_FLAG_LBUTTON",
                34:"EVENT_FLAG_ALTKEY + EVENT_FLAG_RBUTTON", 35:"EVENT_FLAG_ALTKEY + EVENT_FLAG_MBUTTON"}


def nothing(x):
    pass
x_pos_1, y_pos_1 = 0,0
x_pos_2, y_pos_2 = 0,0
unmosaic = False


global x_pos_
global y_pos_
global x_dis_
global y_dis_

x_pos_ = []
y_pos_ = []
x_dis_ = []
y_dis_ = []

i = 0


drawing = False
def mouse_callback(event, x,y, flags, param):

    global i
    global test
    global op

    if event == cv2.EVENT_LBUTTONDOWN: #마우스를 누른 상태
        drawing = True
        x_pos_.append(x)
        y_pos_.append(y)

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        x_pos_.append(x)
        y_pos_.append(y)



    elif event == 8:
        if len(x_pos_)!=0:
            for i in range(len(x_pos_)):
                if min(x_pos_[i],x_pos_[i+1])<= x <= max(x_pos_[i],x_pos_[i+1]):
                    x_pos_[i] = 0
                    y_pos_[i] = 0
                    x_pos_[i+1] = 1
                    y_pos_[i+1] = 1
                    i = i+2






cv2.namedWindow('VideoFrame', cv2.WINDOW_NORMAL)


cv2.setMouseCallback('VideoFrame', mouse_callback)



#dnn part

model = 'res10_300x300_ssd_iter_140000_fp16.caffemodel'
config = 'deploy.prototxt'


#net, cap open check
if not cap.isOpened():
    print('Camera open failed!')
    exit()

net = cv2.dnn.readNet(model, config)

if net.empty():
    print('Net open failed!')
    exit()



#cv2.namedWindow('VideoFrame',)

while True:

#    cv2.resizeWindow('VideoFrame', (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

#    img_blur = copy.deepcopy(img)
    ret, orig_image = cap.read()
    orig_image_tmp = copy.deepcopy(orig_image)

    """
    cap.set(cv2.CAP_PROP_FRAME_WIDTH,cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT,cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    """

    if orig_image is None:
        print("end")
        break



    image = cv2.cvtColor(orig_image, cv2.COLOR_BGR2RGB)



    timer.start()
    boxes, labels, probs = predictor.predict(image, candidate_size / 2, threshold)

    interval = timer.end()


    for i in range(boxes.size(0)):
        box = boxes[i, :]
     #   label = f" {probs[i]:.2f}"
        if box[1] < 0:
            box[1] = 0
        if box[0] < 0:
            box[0] = 0
        orig_image = blur_circle(orig_image, int(box[0]), int(box[1]), int(box[2]), int(box[3]))

        right_count = 0
        for s in range(3):
            face = cv2.resize(orig_image_tmp[int(box[1]):int(box[3]),int(box[0]):int(box[2])],(200,200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

            result = my_model[s].predict(face)
          #  print(result[0]) #닮은 사진.
            print(result[1]) #유사도 낮을수록 유사한 것.
            if result[1] < 80:
#                confidence_my = int(100*(1-(result[1])/300))
                my_face = orig_image_tmp[int(box[1]):int(box[3]),int(box[0]):int(box[2])]
                orig_image[int(box[1]):int(box[3]),int(box[0]):int(box[2])] = my_face
            if result[1]   > 80:
                right_count += 1
        if right_count > 0:
            orig_image = blur_circle(orig_image, int(box[0]), int(box[1]), int(box[2]), int(box[3]))



#            if confidence_my > 77:
#                print('jo')
#                my_face = orig_image_tmp[int(box[1]):int(box[3]),int(box[0]):int(box[2])]
#                orig_image[int(box[1]):int(box[3]),int(box[0]):int(box[2])] = my_face

    if len(x_pos_)!=0:
        for i in range(len(x_pos_)-1):
            if i %2 != 0: continue
            if x_pos_[i] == 0 and y_pos_[i] == 0 and x_pos_[i+1] == 1 and y_pos_[i+1] == 1:
                orig_image = blur_circle(orig_image, x_pos_[i], y_pos_[i], x_pos_[i+1], y_pos_[i+1])
            elif y_pos_[i] > y_pos_[i+1] and x_pos_[i] > x_pos_[i+1]:
                orig_image = blur_circle(orig_image, x_pos_[i+1], y_pos_[i+1], x_pos_[i], y_pos_[i])
            elif y_pos_[i]>y_pos_[i+1] and x_pos_[i] < x_pos_[i+1]:
                orig_image = blur_circle(orig_image, x_pos_[i], y_pos_[i+1], x_pos_[i+1], y_pos_[i])
            elif y_pos_[i]<y_pos_[i+1] and x_pos_[i] > x_pos_[i+1]:
                orig_image = blur_circle(orig_image, x_pos_[i+1], y_pos_[i], x_pos_[i], y_pos_[i+1])
            elif y_pos_[i]<y_pos_[i+1] and x_pos_[i] < x_pos_[i+1]:
                orig_image = blur_circle(orig_image, x_pos_[i], y_pos_[i], x_pos_[i+1], y_pos_[i+1])
    #    unmosaic = False

    sum += boxes.size(0)
    cv2.imshow('VideoFrame', orig_image)
    out.write(orig_image)
    if cv2.waitKey(1) > 0:
        break

cap.release()
out.release()
cv2.destroyAllWindows()

priors nums:4420
Inference time:  0.040004
71.0732552067175
81.50673793542528
82.30023602208554
42.51128886000338
57.35002801868381
62.1227621343305
Inference time:  0.035003
72.5416738529116
77.80798727617102
73.98681390142161
45.89648023237961
57.375029307562116
61.006252451530834
Inference time:  0.039998
47.27192413086976
60.82248054595849
63.24117267035135
73.06588018737474
81.39452936676456
74.8589579532112
Inference time:  0.036004
74.67244110244579
84.62087867493098
75.39587674679797
47.342396190662924
61.97944608899013
65.32193199200779
Inference time:  0.035002
72.58937417351594
83.53988730766969
77.43532437766156
47.51126400964863
61.83859613291573
63.56826358169092
Inference time:  0.034004
49.57095546953575
60.13362544729574
62.76304923224548
71.83975158918047
83.7213347147643
73.31877384478143
Inference time:  0.036999
73.20710606617118
82.29426841382235
76.08706440079621
49.18931387399386
59.469036340431614
64.62534664604142
Inference time:  0.034004
72.05606785678161
78

Inference time:  0.041001
79.92300747007253
84.135377185333
81.12794211251908
48.544682073449
60.3578805739844
66.51545462299073
Inference time:  0.032999
42.8236332034355
58.43633835234373
65.62530154630562
79.77446597019886
90.47229820236538
75.32827701008952
Inference time:  0.033
43.86737109221442
52.86214671522206
66.45836799649666
81.61430027413368
93.35000161557201
75.08208135882145
Inference time:  0.031
48.29034705572613
48.584976747939116
67.89357652822392
75.65704613063026
86.37453190983803
70.7951753736369
Inference time:  0.037
48.35207376427994
45.0431953749214
69.65143082125986
77.67237176745125
88.56145717624794
73.64473101521646
Inference time:  0.035001
50.89172735443181
47.84161197590114
70.31263505067017
77.51319257690469
83.2251851244064
74.41571589225673
Inference time:  0.036001
78.43224927039338
84.17318507184265
71.95083700210266
45.178041702532866
60.05825135629725
68.55119918098677
Inference time:  0.038004
43.61937515118371
59.59777067097701
68.4695687479220

Inference time:  0.031999
44.77313831096069
58.26203243855517
66.80888916055835
74.97865699052188
79.87953885844547
84.41842568680289
Inference time:  0.035003
44.15194797071352
59.89207932071875
67.72516599192801
75.46649145807959
77.75761178590201
86.27428132214469
Inference time:  0.032998
46.56129593793676
58.81011595936255
65.52734399010836
77.02518394765195
79.69437282453544
86.82018538838683
Inference time:  0.032003
45.04398144564299
59.35780025138114
65.08613545747441
85.79267509147245
94.4020282444189
97.34362203859976
Inference time:  0.033002
45.032616554700205
59.90501380467825
65.56325633781738
79.56455044712189
80.04739408578017
86.66283122423398
Inference time:  0.038002
47.0458437861429
58.92311759998011
63.96547779580353
84.92592250735949
84.38209545516563
92.0877199163624
Inference time:  0.034002
49.597548601481144
60.27292838212372
67.47576579039986
77.67498683725167
79.47844793535894
83.78336408532232
Inference time:  0.033002
50.877885243447935
63.958238051457805

Inference time:  0.034001
52.102255886617336
67.26195551364313
61.51934860399976
Inference time:  0.034001
54.24973811626308
68.75922919730532
61.834664970361885
Inference time:  0.033003
51.03946286114138
66.12085312940836
60.681875826064186
Inference time:  0.033001
53.074923438810515
67.14299768062924
59.8290966676795
Inference time:  0.034001
54.771902585565115
69.40316500853366
60.28225285634587
Inference time:  0.031
58.25626941405063
73.05869882338433
60.53473697548001
Inference time:  0.028001
63.033048047178724
74.47236822799366
70.95615300131878
Inference time:  0.029001
57.31158789795143
69.18048937925981
76.12248372766184
Inference time:  0.032
51.45328140411376
62.27167502409514
75.14627392878214
Inference time:  0.032004
48.07895227186794
60.2284270406811
69.10187592628685
Inference time:  0.032004
49.40872701606418
62.00144355198785
64.67982966681045
Inference time:  0.034003
50.268817899676954
65.46113871566203
63.54465204003584
Inference time:  0.037999
49.935160810896

102.69296861281356
Inference time:  0.033001
92.81581724195944
91.09704552663972
95.23864914418559
Inference time:  0.030003
59.90501809623867
64.06014559368167
77.8006787463606
112.7691901388649
107.86150178581188
103.65537691642268
Inference time:  0.035002
50.440940305573434
51.25145615400626
69.4543246460217
99.17977787131835
93.91112892905548
102.3014725467249
Inference time:  0.032002
48.04974806109585
56.94932953156854
64.8811591888612
Inference time:  0.036002
44.34840728436938
58.14769429563237
64.37005823380646
Inference time:  0.031004
44.10966805208273
60.148717816959994
66.71663838474267
Inference time:  0.032999
44.58088136790893
62.44763025788304
67.26123465161986
Inference time:  0.031001
44.29350301611253
60.62693225793886
64.08428575457425
Inference time:  0.033003
41.99612882540783
60.47403872894286
65.27745194962446
Inference time:  0.034002
51.80450337830939
63.9545166023515
74.42518543347131
111.91519439453906
112.44827603623816
102.18237928778947
Inference time: 

IndexError: index 480 is out of bounds for axis 0 with size 480